I am very close to understanding this authentication 
The `class FileOrData` class is the key. This is actually creating the cert files from the 
info in the kubeconfig file. 
Walk through this class to see how it is creating the files to then copy that and see if I can create my own pool_manager 

In [ ]:
from kubernetes import client, config

config.load_kube_config()

client.api_client
c = client.CoreV1Api()

In [ ]:
import base64 
import atexit 
import tempfile

_temp_files = {}
def _create_temp_file_with_content(content):
    if len(_temp_files) == 0:
        atexit.register(_cleanup_temp_files)
    # Because we may change context several times, try to remember files we
    # created and reuse them at a small memory cost.
    content_key = str(content)
    if content_key in _temp_files:
        return _temp_files[content_key]
    _, name = tempfile.mkstemp()
    _temp_files[content_key] = name
    with open(name, "wb") as fd:
        fd.write(content.encode() if isinstance(content, str) else content)
    return name

def _cleanup_temp_files():
    global _temp_files
    for temp_file in _temp_files.values():
        try:
            os.remove(temp_file)
        except OSError:
            pass
    _temp_files = {}

class FileOrData(object):
    """Utility class to read content of obj[%data_key_name] or file's
     content of obj[%file_key_name] and represent it as file or data.
     Note that the data is preferred. The obj[%file_key_name] will be used iff
     obj['%data_key_name'] is not set or empty. Assumption is file content is
     raw data and data field is base64 string. The assumption can be changed
     with base64_file_content flag. If set to False, the content of the file
     will assumed to be base64 and read as is. The default True value will
     result in base64 encode of the file content after read."""
    def __init__(
        self,
        obj,
        file_key_name,
        data_key_name=None,
        file_base_path="",
        base64_file_content=True,
    ):
        if not data_key_name:
            data_key_name = file_key_name + "-data"
        self._file = None
        self._data = None
        self._base64_file_content = base64_file_content
        if data_key_name in obj:
            self._data = obj[data_key_name]
        elif file_key_name in obj:
            self._file = os.path.normpath(
                os.path.join(file_base_path, obj[file_key_name])
            )

    def as_file(self):
        """If obj[%data_key_name] exists, return name of a file with base64
        decoded obj[%data_key_name] content otherwise obj[%file_key_name]."""
        use_data_if_no_file = not self._file and self._data
        if use_data_if_no_file:
            if self._base64_file_content:
                if isinstance(self._data, str):
                    content = self._data.encode()
                else:
                    content = self._data
                self._file = _create_temp_file_with_content(
                    base64.standard_b64decode(content)
                )
            else:
                self._file = _create_temp_file_with_content(self._data)
        if self._file and not os.path.isfile(self._file):
            raise ConfigException("File does not exists: %s" % self._file)
        return self._file

    def as_data(self):
        """If obj[%data_key_name] exists, Return obj[%data_key_name] otherwise
        base64 encoded string of obj[%file_key_name] file content."""
        use_file_if_no_data = not self._data and self._file
        if use_file_if_no_data:
            with open(self._file) as f:
                if self._base64_file_content:
                    self._data = bytes.decode(
                        base64.standard_b64encode(str.encode(f.read()))
                    )
                else:
                    self._data = f.read()
        return self._data

In [ ]:
import os
import yaml

def get_current(config, current_context):
    for i_cluster, i_user in zip(config["clusters"], config["users"]): 
        if i_cluster["name"] == current_context:
            cluster = i_cluster
            user = i_user
            name = cluster["name"]
            combined = {"name": name}
            for k, v in cluster["cluster"].items():
                combined[k] = v
            for k, v in user["user"].items():
                combined[k] = v
            return combined
    else:
        raise Exception("No match!")
    

base_path = os.environ["KUBECONFIG"]
with open(base_path) as f:
    kubeconfig = yaml.safe_load(f)
kubeconfig.keys()

c_obj = get_current(kubeconfig, kubeconfig["current-context"])
c_obj.keys()

In [ ]:
import os 
ssl_ca_cert = FileOrData(c_obj, "certificate-authority", file_base_path=base_path).as_file()


In [ ]:
cert_file = FileOrData(
    c_obj, "client-certificate", file_base_path=base_path
).as_file()
key_file = FileOrData(
    c_obj, "client-key", file_base_path=base_path
).as_file()

In [ ]:
print(type(ssl))

In [ ]:
import urllib3
import ssl
pool_manager = urllib3.PoolManager(
    num_pools=4,
    maxsize=50,
    cert_reqs=ssl.CERT_REQUIRED,
    ca_certs=ssl_ca_cert,
    cert_file=cert_file,
    key_file=key_file
)

In [ ]:
url

In [ ]:
url = c_obj["server"] + "/api/"
headers = {'Accept': 'application/json', 'User-Agent': 'OpenAPI-Generator/12.0.0-snapshot/python', 'Content-Type': 'application/json'}
r = pool_manager.request(
    "GET", 
    url, 
    headers=headers, 
    preload_content=True
)
r

In [ ]:
import json 
d = json.loads(r.data)
d

In [ ]:
dir(r)